In [98]:
import pandas as pd
import numpy as np
from datetime import datetime
import glob, re
from dateutil.parser import parse as date_parse

In [3]:
weather_set_dir = 'jhz-weather-data'
air_store_info = pd.read_csv('{}/air_store_info_with_nearest_active_station.csv'.format(weather_set_dir))
hpg_store_info = pd.read_csv('{}/hpg_store_info_with_nearest_active_station.csv'.format(weather_set_dir))

air_station_distances = pd.read_csv('{}/air_station_distances.csv'.format(weather_set_dir))
hpg_station_distances = pd.read_csv('{}/hpg_station_distances.csv'.format(weather_set_dir))

weather_stations = pd.read_csv('{}/weather_stations.csv'.format(weather_set_dir))
nearby_active_stations = pd.read_csv('{}/nearby_active_stations.csv'.format(weather_set_dir))
feature_manifest = pd.read_csv('{}/feature_manifest.csv'.format(weather_set_dir))

In [67]:
# Read all the csv files
# Note: need to parse date into datetime type
dfs = { re.search('/([^/\.]*)\.csv', fn).group(1):
    pd.read_csv(fn, parse_dates=['calendar_date'])for fn in glob.glob('1-1-16_5-31-17_Weather/*.csv')}
for k, v in dfs.items(): locals()[k] = v

In [12]:
dfs['aichi__ai-xi-kana__isaai'].head()

,calendar_date,avg_temperature,high_temperature,low_temperature,precipitation,hours_sunlight,solar_radiation,deepest_snowfall,total_snowfall,avg_wind_speed,avg_vapor_pressure,avg_local_pressure,avg_humidity,avg_sea_pressure,cloud_cover
0,2016-01-01,6.0,11.0,0.7,0.0,9.1,NaN,NaN,NaN,2.9,NaN,NaN,NaN,NaN,NaN
1,2016-01-02,4.7,10.5,0.0,0.0,6.8,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN
2,2016-01-03,7.0,13.8,1.9,0.0,8.6,NaN,NaN,NaN,1.4,NaN,NaN,NaN,NaN,NaN
3,2016-01-04,8.8,14.7,2.7,0.0,5.3,NaN,NaN,NaN,2.2,NaN,NaN,NaN,NaN,NaN
4,2016-01-05,8.9,14.1,3.4,0.0,4.5,NaN,NaN,NaN,2.7,NaN,NaN,NaN,NaN,NaN


In [17]:
def select_stations(latitude_str, longitude_str, distance_df, effective_range=20.0, date_floor=None, top_n=5):
    """
    Filters stations based on proximity to coordinates, and termination status
    Note: if longitude_str is None, the first argument is assumed to be a properly formatted coordinate string
    :param latitude_str: latitude_str from air/hpg_store_info_with_nearest_active_station
    :param longitude_str: longitude_str from air/hpg_store_info_with_nearest_active_station
    :param distance_df: one of the following DFs: air_station_distances, hpg_station_distances
    :param effective_range: float in kilometers specifying the max distance a station can be from the store
    :param date_floor: if datetime, remove stations terminated before date_floor. If None, ignore termination
    :param top_n: if int, return at most top_n many stations. If None, all stations will be returned
    :returns: a list of tuples of (station_id, distance) that meet the given specifications - sorted by distance
    """
    if longitude_str is not None:
        _lookup_coords = '({}, {})'.format(latitude_str, longitude_str).replace('"', '')
    else:
        _lookup_coords = latitude_str
        
    _ids, _distances = distance_df['station_id'].values, distance_df[_lookup_coords]
    _result = [(_ids[_], _distances[_]) for _ in range(len(_ids)) if _distances[_] <= effective_range]
    
    if date_floor is not None and isinstance(date_floor, datetime):
        _result = [_ for _ in _result if '____' not in _[0] or date_parse(_[0].split('____')[1]) > date_floor]

    return sorted(_result, key=lambda _: _[1])[:top_n]


In [19]:
store = hpg_store_info.iloc[0]
lat_str, lon_str = store['latitude_str'], store['longitude_str']
_test_0 = select_stations(lat_str, lon_str, hpg_station_distances)

In [23]:
_test_1 = select_stations(lat_str, lon_str, hpg_station_distances, date_floor=date_parse('2017-5-31'))

In [24]:
_test_1

[('tokyo__setagaya-kana__setagaya', 4.7579321766000007),
 ('tokyo__tokyo-kana__tonokyo', 9.1201964878599995),
 ('kanagawa__hiyoshi-kana__hiyoshi', 10.341102465300001),
 ('tokyo__nerima-kana__nerima', 12.582557308399998),
 ('tokyo__haneda-kana__haneda', 14.250354447699999)]

In [104]:
start_date = datetime(2016,1,1)
end_date = datetime(2017,5,31)

In [105]:
pd.date_range(start_date, end_date)

DatetimeIndex(['2016-01-01', '2016-01-02', '2016-01-03', '2016-01-04',
               '2016-01-05', '2016-01-06', '2016-01-07', '2016-01-08',
               '2016-01-09', '2016-01-10',
               ...
               '2017-05-22', '2017-05-23', '2017-05-24', '2017-05-25',
               '2017-05-26', '2017-05-27', '2017-05-28', '2017-05-29',
               '2017-05-30', '2017-05-31'],
              dtype='datetime64[ns]', length=517, freq='D')

In [103]:
df.iloc[517]

air_store_id     hpg_store_id                      hpg_6622b623...
calendar_date                                  2017-06-01 00:00:00
Name: 517, dtype: object

In [113]:
store

hpg_store_id                      hpg_6622b62385aec8bf
hpg_genre_name                          Japanese style
hpg_area_name            Tōkyō-to Setagaya-ku Taishidō
latitude                                       35.6437
longitude                                      139.668
latitude_str                        "35.6436746642265"
longitude_str                       "139.668220854814"
station_id              tokyo__setagaya-kana__setagaya
station_latitude                               35.6267
station_longitude                               139.62
station_vincenty                               4.75793
station_great_circle                           4.75187
Name: 0, dtype: object

In [131]:
# For each store, get their top 5 stations within 20 kms
# Add a date column for each store
# Search the station files, merge on these dates
def get_weather(store_id):
    dates = pd.date_range(start_date, end_date)
    length = len(dates)
    df = pd.DataFrame({'air_store_id': [store_id] * len(dates), 'calendar_date': dates})
    selected_stations = select_stations(lat_str, lon_str, hpg_station_distances, date_floor=date_parse('2017-5-31'))
    # Skip two snow features
    weather_columns = [
        'avg_temperature', 'high_temperature', 'low_temperature', 'precipitation',
        'hours_sunlight', 'solar_radiation',  'avg_wind_speed',
        'avg_vapor_pressure', 'avg_local_pressure', 'avg_humidity', 'avg_sea_pressure', 
        'cloud_cover'
    ]
    
    weather_df = [dfs[station[0]].loc[(dfs[station[0]].calendar_date >= start_date) & (dfs[station[0]].calendar_date < end_date)] for station in selected_stations]

    for col in weather_columns:
        temp = []
        for i in range(length):
            total, count = 0, 0
            for station_df in weather_df:
                try:
                    val = station_df.iloc[i][col]
                except:
                    None
                if not np.isnan(val):
                    total += val
                    count += 1
            if count > 0:
                temp.append(total/count*1.0)
            else:
                temp.append(np.nan)
        df[col] = temp
    df = df.fillna(method='ffill')
    return df
            
   
    

In [125]:
store_id = hpg_store_info.iloc[2].hpg_store_id
df = get_weather(store_id)

In [127]:
weather_columns = [
        'avg_temperature', 'high_temperature', 'low_temperature', 'precipitation',
        'hours_sunlight', 'solar_radiation',  'avg_wind_speed',
        'avg_vapor_pressure', 'avg_local_pressure', 'avg_humidity', 'avg_sea_pressure', 
        'cloud_cover'
    ]
for col in weather_columns:
    df = df.fillna(method='ffill')
    print(col, df[col].isnull().sum()/len(df))

avg_temperature 0.0
high_temperature 0.0
low_temperature 0.0
precipitation 0.0
hours_sunlight 0.0
solar_radiation 0.0
avg_wind_speed 0.0
avg_vapor_pressure 0.0
avg_local_pressure 0.0
avg_humidity 0.0
avg_sea_pressure 0.0
cloud_cover 0.0


In [129]:
air = pd.read_csv('input/air_store_info.csv')
air.head()

,air_store_id,air_genre_name,air_area_name,latitude,longitude
0,air_0f0cdeee6c9bf3d7,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
1,air_7cc17a324ae5c7dc,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
2,air_fee8dcf4d619598e,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
3,air_a17f0778617c76e2,Italian/French,Hyōgo-ken Kōbe-shi Kumoidōri,34.695124,135.197852
4,air_83db5aff8f50478e,Italian/French,Tōkyō-to Minato-ku Shibakōen,35.658068,139.751599


In [132]:
df_list = []
for store_id in air.air_store_id.values:
    df = get_weather(store_id)
    df_list.append(df)
weather_store = pd.concat(df_list)
    
    

In [137]:
weather_store.isnull().any()

air_store_id          False
calendar_date         False
avg_temperature       False
high_temperature      False
low_temperature       False
precipitation         False
hours_sunlight        False
solar_radiation       False
avg_wind_speed        False
avg_vapor_pressure    False
avg_local_pressure    False
avg_humidity          False
avg_sea_pressure      False
cloud_cover           False
dtype: bool

In [138]:
weather_store.to_csv("input/air_store_weather.csv",index=False)